In [1]:
import warnings
from sklearn.externals import joblib
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.decomposition import PCA
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore")

/home/alan/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# 特征处理

In [4]:
df = pd.read_csv(r'../Data/TRAINSET_STOCK.csv')
df_add1 = pd.read_csv(r'../Data/20190506_STOCK.csv')
df_add2 = pd.read_csv(r'../Data/20190514_STOCK.csv')
df_add3 = pd.read_csv(r'../Data/20190521_STOCK.csv')
df_add4 = pd.read_csv(r'../Data/20190528_STOCK.csv')
df_copy = pd.concat([df, df_add1, df_add2, df_add3, df_add4])
df_copy = df[~df.index.duplicated()]

In [23]:
df = df_copy.copy()

In [18]:
areas = ['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
         '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
         '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
         '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
         '交银装备']

areas_map = {}
for i, v in enumerate(areas):
    areas_map[v] = i

In [24]:
ts_code = list(df['ts_code'].unique())
code_map = {}
for i, v in enumerate(ts_code):
    code_map[v] = i

In [27]:
df.head()

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,0,20140401,农林牧渔,1668.75,1668.54,1689.12,1689.07,22.13,1.33,34914.0,291113.0,41.51,2.77,1
1,0,20140402,农林牧渔,1688.72,1684.53,1693.41,1692.24,3.17,0.19,36300.0,289020.0,41.63,2.79,1
2,0,20140403,农林牧渔,1693.05,1679.85,1697.73,1685.71,-6.53,-0.39,31403.0,259464.0,41.38,2.78,0
3,0,20140404,农林牧渔,1681.92,1680.34,1698.44,1698.25,12.54,0.74,28648.0,240940.0,41.76,2.80,1
4,0,20140408,农林牧渔,1693.24,1692.22,1706.84,1706.84,8.59,0.51,35012.0,312423.0,42.00,2.79,1


In [26]:
df.ts_code = df.ts_code.map(code_map).astype(int)

In [4]:
x = []
y = []
for arr in columns:
    df1 = df.loc[df['name'] == arr]
    df1 = df1[['ts_code', 'trade_date', 'name', 'low',
               'high', 'close', 'pct_change', 'vol', 'y']]
    df1['vol_change'] = df1['vol'].pct_change()
    df1['low_change'] = df1['low'].pct_change()
    df1['high_change'] = df1['high'].pct_change()
    df1['gaodi'] = (df1['high'] - df1['low']) / df1['low']
    df1['gaoclo'] = (df1['high'] - df1['close']) / (df1['high'] - df1['low'])
    df1['diclo'] = (df1['close'] - df1['low']) / (df1['high'] - df1['low'])
    df1['pct_change'] = df1['pct_change'] / 10
    df1 = df1[['pct_change', 'vol_change', 'high_change',
               'low_change', 'gaodi', 'gaoclo', 'diclo', 'y']]
    df1 = df1.dropna()
    dataset = df1.values
    for i in range(len(dataset) - 15):
        x.append(dataset[i:(i + 10), :8].tolist())
        y.append(dataset[(i + 10):(i + 15), 7, np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=30)
xtrain, xtest, ytrain, ytest = np.array(xtrain), np.array(
    xtest), np.array(ytrain), np.array(ytest)
# xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1]*xtrain.shape[2], 1))
# xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest = np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))

In [28]:
df1 = df[['ts_code', 'trade_date', 'name', 'low',
          'high', 'close', 'pct_change', 'vol', 'y']]
df1['vol_change'] = df1['vol'].pct_change()
df1['low_change'] = df1['low'].pct_change()
df1['high_change'] = df1['high'].pct_change()
df1['gaodi'] = (df1['high'] - df1['low']) / df1['low']
df1['gaoclo'] = (df1['high'] - df1['close']) / (df1['high'] - df1['low'])
df1['diclo'] = (df1['close'] - df1['low']) / (df1['high'] - df1['low'])
df1['pct_change'] = df1['pct_change'] / 10
df1 = df1[['ts_code', 'pct_change', 'vol_change', 'high_change',
           'low_change', 'gaodi', 'gaoclo', 'diclo', 'y']]
df1 = df1.dropna()

In [30]:
df1.head(), df1.shape

(   ts_code  pct_change  vol_change  high_change  low_change     gaodi  \
 1        0       0.019    0.039698     0.002540    0.009583  0.005272   
 2        0      -0.039   -0.134904     0.002551   -0.002778  0.010644   
 3        0       0.074   -0.087730     0.000418    0.000292  0.010772   
 4        0       0.051    0.222145     0.004946    0.007070  0.008640   
 5        0       0.110    0.231406     0.011231    0.009024  0.010846   
 
      gaoclo     diclo  y  
 1  0.131757  0.868243  1  
 2  0.672260  0.327740  0  
 3  0.010497  0.989503  1  
 4  0.000000  1.000000  1  
 5  0.024838  0.975162  1  , (39167, 9))

In [5]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((30579, 10, 8), (7645, 10, 8), (30579, 5), (7645, 5))